In [2]:
# Get environment variables
from dotenv import load_dotenv
import os

load_dotenv()  # Load environment variables from .env file

secret = os.getenv('SMARTLEAD_API_KEY')
openai_secret = os.getenv('OPENAI_KEY')
apollo_secret = os.getenv('APOLLO_API_KEY')


In [5]:
# Import dependencies
import requests
import csv
import json
import html2text
import ast
from openai import OpenAI as OpenAIPython
from langchain.llms import OpenAI
from langchain.schema import SystemMessage
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferWindowMemory, ConversationSummaryBufferMemory
from langchain.agents import initialize_agent, AgentType
from langchain_openai  import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

OpenAI_LLM = OpenAI(temperature=0.6,api_key=openai_secret)
ChatOpenAI_LLM = ChatOpenAI(temperature=0, model="gpt-4o",api_key=openai_secret)
client = OpenAIPython(
    # This is the default and can be omitted
    api_key=openai_secret,
)
from datetime import datetime
current_datetime = datetime.utcnow()

In [4]:
# Get all the campaigns
response = requests.get(f"https://server.smartlead.ai/api/v1/campaigns?api_key={secret}")
response = response.json()
print(response)

KeyboardInterrupt: 

In [ ]:
from pydantic import BaseModel, Field
from typing import Type, List
from langchain.tools import BaseTool
# Input class for tool so that it can follow strict input parameter schema
class CategorizeEmailInput(BaseModel):
    conversation: str = Field(description="Email conversation array")

class CategorizeEmailTool(BaseTool):
		# Provide proper name and description for your tool
    name: str = "email_categorizer_tool"
    description: str = "use this tool when have email conversation history and you want to categorize this email"
    args_schema: Type[BaseModel] = CategorizeEmailInput

    def _run(self, conversation: str):
      prompt = f"""
        Email Conversation History:
        ---
        {conversation}
        ---
        You have given an array of conversations between Jamie and a client.
        Your goal is to categorize this email based on the conversation history from the given categories:

        1. Interested_Needs_Further_Discussion: Prospects who show positive engagement with your offering, asking questions or expressing clear interest in learning more.
        2. Request_For_More_Information: When a prospect asks for additional details about your product or service to make an informed decision.
        3. Positive_Feedback: Emails where the recipient expresses appreciation for your outreach or provides positive comments about your company.
        4. Open_To_A_Call: When the prospect indicates they're willing to schedule a call to discuss further.
        5. Needs_Immediate_Follow_Up: Urgent situations where a prospect needs a quick response due to a time-sensitive need.
        6. Not_Interested_Politely_Decline: Prospects who clearly state they are not interested in your offering but do so in a respectful manner.
        7. Out_Of_Scope: Emails from individuals or companies that fall outside your target market or service area.
        8. Wrong_Contact: Emails sent to the wrong person within a company or an outdated contact.
        9. Auto_Reply: Generic replies indicating the recipient is out of office or not currently monitoring their email.
        10. Spam_Indicator: Emails with suspicious subject lines, excessive exclamation points, or other signs of spam.
        11. Negative_Response: Aggressive or hostile replies, personal attacks, or clear disinterest in your offering.
        12. Unsubscribed: Individuals who have opted out of receiving further communication from your company.
        13. Info: These are emails that don't fit into any of the above categories.

        Note: Your final response MUST BE the category name ONLY

        RESPONSE:
      """
      message = client.chat.completions.create(
          model="gpt-4o-mini",
          messages=[
              {"role": "user", "content": prompt}
          ]
      )
      category = message.choices[0].message.content
      return category

    def _arun(self, url: str):
        raise NotImplementedError(
            "categorise_email does not support async")

In [ ]:
class EmailWriterToolInput(BaseModel):
    latest_reply: str = Field(description="Latest reply from the prospect")
    conversation_history: str = Field(description="Array of conversation history")
    sender: str = Field(description="Name of sender")

class EmailWriterTool(BaseTool):
    name: str = "email_writer_tool"
    description: str = "Use this tool to construct a reply to an email."
    args_schema: Type[BaseModel] = EmailWriterToolInput

    def __init__(self):
        super().__init__()

        # Load past emails and FAQs into ChromaDB
        embedding_function = OpenAIEmbeddings()

        # Load past replies
        email_loader = CSVLoader("./final.csv", encoding="utf-8")
        email_documents = email_loader.load()

        # Load FAQs
        faq_loader = CSVLoader("./faq.csv", encoding="utf-8")
        faq_documents = faq_loader.load()

        # Store in ChromaDB
        self.email_db = Chroma.from_documents(email_documents, embedding_function)
        self.faq_db = Chroma.from_documents(faq_documents, embedding_function)

    def retrieve_relevant_knowledge(self, query: str):
        """Retrieve past emails & FAQs relevant to the latest reply."""
        past_emails = self.email_db.similarity_search(query, k=3)
        faqs = self.faq_db.similarity_search(query, k=3)

        past_emails_text = "\n".join([doc.page_content for doc in past_emails])
        faqs_text = "\n".join([doc.page_content for doc in faqs])

        return past_emails_text, faqs_text

    def _run(self, latest_reply: str, conversation_history: str, sender: str):
        # Retrieve relevant past emails & FAQs
        past_emails, faqs = self.retrieve_relevant_knowledge(latest_reply)

        prompt = f"""
        You are the email inbox manager for Jamie, a recruiter at Hyred.
        He has sent cold emails to prospects, and they have replied.
        Your task is to draft responses that mimic Jamie’s past replies.

        **PAST EMAIL EXAMPLES:**
        {past_emails}

        **USE THIS AS YOUR KNOWLEDGE BASE:**
        {faqs}

        **CONVERSATION HISTORY:**
        {conversation_history}

        **NEW EMAIL FROM PROSPECT:**
        {latest_reply}

        **SENDER NAME:** {sender}

        **INSTRUCTIONS:**
        - If the email from prospect cannot be found in the knowledge base, DON'T construct the reply
        - Reply in the same style Jamie used in his past emails.
        - Avoid repeating what the client said.
        - Do not defer topics to another email—respond fully now.
        - Keep it short and easy to understand.
        """

        message = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}]
        )

        return message.choices[0].message.content

    def _arun(self, url: str):
        raise NotImplementedError("Email writer tool does not support async.")

In [6]:
def EmailSenderTool(campaign_id,email_stats_id,email_body,reply_message_id,reply_email_body,email):
  url = f"https://server.smartlead.ai/api/v1/campaigns/{campaign_id}/reply-email-thread?api_key={secret}"
  data = {
    "email_stats_id": email_stats_id,
    "email_body": email_body,
    "reply_message_id": reply_message_id,
    "reply_email_time": current_datetime.strftime('%Y-%m-%dT%H:%M:%S.%fZ'),
    "reply_email_body": reply_email_body,
    "to_email": email,
  }
    
  # Send the request
  try:
      response = requests.post(url, json=data)  # Use `json=data` for JSON payloads
      response.raise_for_status()  # Raise an error for bad status codes (4xx, 5xx)
      response_data = response.json()  # Parse the JSON response
      print("Response:", response_data)
      return response_data
  except requests.exceptions.RequestException as e:
      print(f"An error occurred: {e}")
      return None

In [ ]:

# Creating agent
system_message = SystemMessage(
    content="""
    You are an email inbox assistant of an Jamie who is part of the recruitment team at Hyred,
    Which provides Recruitment services to technical and non-technical organizations
    Jamie have sent a cold email to some leads and you have provided a conversation history between info mation (Jamie's email) and the lead

    Follow these steps while generating email reply:
    Step-1: First categorize the email based on given conversation history and get the category of email.
    Step-2: Check the sender of the last message and if the sender is NOT info mation then goto step-3.
    If the sender of last message is info mation then you don't need to construct a reply

    Step-3: Once you get the category, follow these conditions while constructing a reply email:
    1. If the category is: 
        - Interested_Needs_Further_Discussion
        - Request_For_More_Information
        - Positive_Feedback
        - Open_To_A_Call
        - Needs_Immediate_Follow_Up, you construct the reply email
    Please construct a reply.
    2. For all the other categories, DON'T construct a reply

    Your final response MUST BE in json with these keys:
        {
        "reply": "Constructed email reply for a positive email. Leave this blank if no reply is constructed, or if the last sender is Jamie.",
        "category": "Category of the given email based on the email conversation history. If the 'reply' key is left blank, modify this to 'Manual_Response'."
        }
    RESPONSE(Do not include any extra text, markdown, or formatting. Only return the JSON object.):
    """
)
agent_kwargs = {
    "system_message": system_message,
}
memory = ConversationBufferWindowMemory(
    memory_key='memory',
    k=1,
    return_messages=True
)

tools = [
    CategorizeEmailTool(),
    EmailWriterTool()
]

agent = initialize_agent(
    tools,
    llm=ChatOpenAI_LLM,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    agent_kwargs=agent_kwargs,
    memory=memory,
    handle_parsing_errors=True
)


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_1420\3862083426.py:27: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_1420\3862083426.py:38: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [14]:
# Run the agent for your leads
campaign_id = 1501858
with open('Campaign_Leads.csv',encoding='utf-8', mode='r') as file:
  # Create a CSV reader object
  csv_reader = csv.reader(file)
  for index,row in enumerate(csv_reader):
    if index == 0:
      continue
    # Get lead ID
    lead_info = requests.get(f"https://server.smartlead.ai/api/v1/leads/?api_key={secret}&email={row[2]}")
    lead_info = lead_info.json()
    # Get conversation history of lead
    message_history = requests.get(f"https://server.smartlead.ai/api/v1/campaigns/{campaign_id}/leads/{lead_info['id']}/message-history?api_key={secret}")
    message_history = message_history.json()
    message_history = message_history["history"]
    # print(message_history)
    conversation_history = []
    # Format every message in conversation history
    for message in message_history:
      plain_text = html2text.html2text(message["email_body"])
      # print(message["email_body"])
      prompt = f"""
        Email Thread:
        ---
        {plain_text}
        ---
        You have given a email thread as a plain text and you have to return the latest email from it

        You have to follow these steps to do it:
        Step-1: Get the text which don't starts with '>' because every other text which starts with '>' is a old message in thread
        (It will be at the starting of text and older messages will be at below this latest message)

        The email thread format typically look like this:
        ---
        Hey john,
        I am interested
        Thanks,
        shivam
        > sentence 1 ....
        > sentence 2 ....
        > ...
        >> sentence from older emails ...
        >> sentence from older emails ....
        >> ....
        ---
        For above example, the email message content will be:
        Hey john,
        I am interested
        Thanks,
        Shivam

        Note: Please note that the above message content was just an example and your respond must be related to given plain text

        Step-2: Once you got the latest email then get the message content from it and remove any extra blank space or unnecessary content from email
        After these steps, Return the email message content in response

        RESPONSE (Don't return anything except email message):
      """
      email_content = client.chat.completions.create(
          model="gpt-4o-mini",
          messages=[
              {"role": "user", "content": prompt}
          ]
      )
      email_content = email_content.choices[0].message.content
      convo = {
          "sender": "info mation" if message["type"] == "SENT" else row[0],
          "message": email_content
      }
      conversation_history.append(convo)
      # print(convo)
    # Prompt for our agent
    prompt = f"""
      Email conversation history:
      ---
      {conversation_history}
      ---
      Lead Name: {row[0]}
      Lead Email: {row[2]}

      Sender of last message: {conversation_history[len(conversation_history) - 1]["sender"]}
    """
    # print(prompt)
    response = agent({"input": prompt})
    response = json.loads(response["output"])
    history = message_history[len(message_history)-1]
	  # If there is reply which needs to be send then use email sender tool to send email
    if response['reply'] != "":
    	res = EmailSenderTool(campaign_id=campaign_id,email_stats_id=history["stats_id"],email_body=response['reply'],reply_message_id=history["message_id"],reply_email_body=history["email_body"],email=row[2])



> Entering new AgentExecutor chain...

Invoking: `email_categorizer_tool` with `{'conversation': "[{'sender': 'info mation', 'message': 'Hi Trang,\\n\\nI’m Ron, CEO at Hyred, and I’m reaching out to see if you’re looking to build a top-tier team in Southeast Asia or Hong Kong. Recruitment here comes with unique challenges—but we make it seamless.\\n\\nFor 18 years, we’ve been helping companies like yours not just fill roles, but find the perfect fit—people who align with your vision, culture, and goals.\\n\\nHere’s how we make hiring easier for you:\\n\\n✔️ Local expertise – We know these markets inside out.\\n\\n✔️ Tailored approach – We adapt to your hiring needs.\\n\\n✔️ Culture-first mindset – Skills matter, but fit is everything.\\n\\nWould you be open to a quick chat to share with us your hiring challenges and explore how we can help you build a talented team?\\n\\nLooking forward to connecting!\\n\\nBest,\\n\\nRon\\n\\nHyred Recruitment Team'}]"}`


Meeting_Ready_Lead{"reply":

In [12]:
print(response)

{'reply': 'Subject: Re: Building a Top-Tier Team\n\nHi Vu,\n\nThanks for getting back to me! I appreciate your response. I’d love to chat about your hiring needs, especially the unique challenges you’re facing in Southeast Asia and Hong Kong.\n\nLet’s schedule a quick call to dive deeper into how we can support you in building that stellar team.\n\nLooking forward to connecting!\n\nBest,  \nRon  \nHyred Recruitment Team', 'category': 'Meeting_Ready_Lead'}
